In [44]:
from datetime import datetime,date
import json
from bs4 import BeautifulSoup
import requests
import dataclasses
from pydantic.dataclasses import dataclass
import copy

In [42]:
# -------------------------------------------
# Modify the holiday class to 
# 1. Only accept Datetime objects for date.
# 2. You may need to add additional functions
# 3. You may drop the init if you are using @dataclasses
# --------------------------------------------
@dataclass
class Holiday:
    name: str
    date: datetime
    
    def __str__ (self):
        return self.name
        # String output
        # Holiday output when printed.

In [47]:
def getHTML(url):
    response = requests.get(url)
    return response.text


In [74]:

bbox = '40.933248,-74.041919,40.596293,-73.757356'

In [75]:
weather_response = requests.get(f'https://www.ncei.noaa.gov/access/services/data/v1/?dataset=daily-summaries&stations=USC00457180,USC00390043&startDate=2020-01-01&endDate=2022-12-12&includeAttributes=true&format=json&bbox={bbox}')

In [ ]:
'https://www.ncei.noaa.gov/access/services/data/v1?dataset=global-summary-of-the-year&dataTypes=DP01,DP05,DP10,DSND,DSNW,DT00,DT32,DX32,DX70,DX90,SNOW,PRCP&stations=ASN00084027&startDate=1952-01-01&endDate=1970-12-31&includeAttributes=true&format=json&includeStationName=true&includeStationLocation=1'

In [76]:
weather_response.json()

[{'DATE': '2020-01-01',
  'STATION': 'USC00457180',
  'PRCP_ATTRIBUTES': ',,7,0700',
  'TMAX': '   61',
  'TMAX_ATTRIBUTES': ',,7',
  'TMIN': '  -33',
  'PRCP': '   99',
  'TOBS': '   44',
  'TOBS_ATTRIBUTES': ',,7,0700',
  'TMIN_ATTRIBUTES': ',,7'},
 {'DATE': '2020-01-02',
  'STATION': 'USC00457180',
  'PRCP_ATTRIBUTES': ',,7,0700',
  'TMAX': '   78',
  'TMAX_ATTRIBUTES': ',,7',
  'TMIN': '    0',
  'PRCP': '    0',
  'TOBS': '    0',
  'TOBS_ATTRIBUTES': ',,7,0700',
  'TMIN_ATTRIBUTES': ',,7'},
 {'DATE': '2020-01-03',
  'STATION': 'USC00457180',
  'PRCP_ATTRIBUTES': ',,7,0700',
  'TMAX': '   50',
  'TMAX_ATTRIBUTES': ',,7',
  'TMIN': '    0',
  'PRCP': '   23',
  'TOBS': '   39',
  'TOBS_ATTRIBUTES': ',,7,0700',
  'TMIN_ATTRIBUTES': ',,7'},
 {'DATE': '2020-01-04',
  'STATION': 'USC00457180',
  'PRCP_ATTRIBUTES': ',,7,0700',
  'TMAX': '   94',
  'TMAX_ATTRIBUTES': ',,7',
  'TMIN': '   33',
  'PRCP': '    0',
  'TOBS': '   33',
  'TOBS_ATTRIBUTES': ',,7,0700',
  'TMIN_ATTRIBUTES': ',,7

In [ ]:

html = requests.get("https://www.timeanddate.com/holidays/us/").text
soup = BeautifulSoup(html, 'html.parser')
table = soup.find('table', attrs={'id':'holidays-table'})
tbody = table.find('tbody')

In [ ]:
e1 = tbody.find('th', attrs={'class':'nw'})

In [ ]:
print(example)

In [ ]:
example.text

In [ ]:
e2 = tbody.find('a')

In [ ]:
type(date(2020,1,3))

In [ ]:
datetime.strptime("Jan 1 2020", "%b %d %Y")

In [ ]:
holidays = []

In [ ]:
holiday_list = []
year = 2022
for date_, holiday_ in zip(tbody.find_all('th'),tbody.find_all('a')):
    fDate = datetime.strptime((date_.text + " " + str(year)),"%b %d %Y").date()
    temp_holiday = Holiday(date = fDate,name = holiday_.text)
    if temp_holiday not in holiday_list:
        holiday_list.append(temp_holiday)

In [ ]:
for row in tbody.find_all('a'):
    print(row.text)

In [45]:
# -------------------------------------------
# The HolidayList class acts as a wrapper and container
# For the list of holidays
# Each method has pseudo-code instructions
# --------------------------------------------
@dataclass
class HolidayList:
    innerHolidays: list[Holiday] = dataclasses.field(default_factory=list)
   
    def addHoliday(self, holidayObj):
        if isinstance(holidayObj, Holiday):
            if holidayObj not in self.innerHolidays:
                print(f"Success:\n {holidayObj} ({holidayObj.date}) has been added to the holiday list.")
                self.innerHolidays.append(holidayObj)
            else:
                print(f'{holidayObj} is already in holiday list')
        else:
            print(f"{holidayObj} is not a Holiday Object")
        # Make sure holidayObj is an Holiday Object by checking the type
        # Use innerHolidays.append(holidayObj) to add holiday
        # print to the user that you added a holiday

    def findHoliday(self, HolidayName, Date, output = True):
        for holiday in self.innerHolidays:
            if HolidayName == holiday.name and Date == holiday.date:
                return holiday
        if output:
            print(f'{HolidayName} and {Date} not in list of holidays')
        return None
        # Find Holiday in innerHolidays
        # Return Holiday

    def removeHoliday(self, HolidayName, Date):
        for i in range(len(self.innerHolidays)):
            if HolidayName == self.innerHolidays[i].name and Date == self.innerHolidays[i].date:
                removed = self.innerHolidays.pop(i)
                print(f"{removed} has been removed from the list")
        # Find Holiday in innerHolidays by searching the name and date combination.
        # remove the Holiday from innerHolidays
        # inform user you deleted the holiday

    def read_json(self, filelocation):
        with open(filelocation, 'r') as f:
            data = json.load(f)
        for hol in data['holidays']:
            tmpDate = datetime.strptime(hol['date'],"%Y-%m-%d").date()
            tmpName = hol['name']
            tmpHoliday = Holiday(tmpName,tmpDate)
            self.addHoliday(tmpHoliday)
        # Read in things from json file location
        # Use addHoliday function to add holidays to inner list.

    def save_to_json(self, filelocation):
        holiday_dict = {"holidays":[{'name':hol.name, 'date':hol.date.strftime('%Y-%m-%d')} for hol in self.innerHolidays]}
        with open(filelocation, 'w') as f:
            jsonHolidays = json.dump(holiday_dict, f, indent=4)
        # Write out json file to selected file.
        
    def scrapeHolidays(self):
        for year in range(2020,2025):
            html = requests.get(f"https://www.timeanddate.com/holidays/us/{year}").text
            soup = BeautifulSoup(html, 'html.parser')
            table = soup.find('table', attrs={'id':'holidays-table'})
            tbody = table.find('tbody')
            for date_, holiday_ in zip(tbody.find_all('th'),tbody.find_all('a')):
                fDate = datetime.strptime((date_.text + " " + str(year)),"%b %d %Y").date()
                temp_holiday = Holiday(date = fDate,name = holiday_.text)
                if temp_holiday not in self.innerHolidays:
                    self.addHoliday(temp_holiday)
        # Scrape Holidays from https://www.timeanddate.com/holidays/us/ 
        # Remember, 2 previous years, current year, and 2  years into the future. You can scrape multiple years by adding year to the timeanddate URL. For example https://www.timeanddate.com/holidays/us/2022
        # Check to see if name and date of holiday is in innerHolidays array
        # Add non-duplicates to innerHolidays
        # Handle any exceptions.     

    def numHolidays(self):
        return len(self.innerHolidays)
        # Return the total number of holidays in innerHolidays
    
    def filter_holidays_by_week(self, year, week_number):
        holidays = sorted(list(filter(lambda x: (x.date.isocalendar()[1] == week_number and x.date.year == year) , self.innerHolidays)), key = lambda x: x.date)
        if week_number == 1:
            range = len(holidays)
            i = 0
            while i < range:
                if holidays[i].date.month == 12:
                    removed = holidays.pop(i)
                    range -= 1
                i += 1
        elif week_number == 52:
            range = len(holidays)
            i = 0
            while i < range:
                if holidays[i].date.month == 1:
                    removed = holidays.pop(i)
                    range -= 1
                i += 1
        return holidays
        # Use a Lambda function to filter by week number and save this as holidays, use the filter on innerHolidays
        # Week number is part of the the Datetime object
        # Cast filter results as list
        # return your holidays

    def displayHolidaysInWeek(self, holidayList):
        for hol in holidayList:
            print(hol)
        # Use your filter_holidays_by_week to get list of holidays within a week as a parameter
        # Output formated holidays in the week. 
        # * Remember to use the holiday __str__ method.

    def getWeather(self, weekNum):
        pass
        # Convert weekNum to range between two days
        # Use Try / Except to catch problems
        # Query API for weather in that week range
        # Format weather information and return weather string.

    def viewCurrentWeek(self):
        year = datetime.now().year
        week = datetime.now().isocalendar()[1]
        current = self.filter_holidays_by_week(year,week)
        self.displayHolidaysInWeek(current)
        # Use the Datetime Module to look up current week and year
        # Use your filter_holidays_by_week function to get the list of holidays 
        # for the current week/year
        # Use your displayHolidaysInWeek function to display the holidays in the week
        # Ask user if they want to get the weather
        # If yes, use your getWeather function and display results

In [ ]:
datetime.now().isocalendar()[1]

In [7]:
hInit = HolidayList()
hInit.scrapeHolidays()
hInit.read_json('example.json')

Success:
 New Year's Day (2020-01-01) has been added to the holiday list.
Success:
 World Braille Day (2020-01-04) has been added to the holiday list.
Success:
 Epiphany (2020-01-06) has been added to the holiday list.
Success:
 Asarah B'Tevet (2020-01-07) has been added to the holiday list.
Success:
 Orthodox Christmas Day (2020-01-07) has been added to the holiday list.
Success:
 International Programmers' Day (2020-01-07) has been added to the holiday list.
Success:
 Estelle Reel Day (2020-01-07) has been added to the holiday list.
Success:
 Battle of New Orleans (2020-01-08) has been added to the holiday list.
Success:
 Stephen Foster Memorial Day (2020-01-13) has been added to the holiday list.
Success:
 Orthodox New Year (2020-01-14) has been added to the holiday list.
Success:
 Lee-Jackson Day (2020-01-17) has been added to the holiday list.
Success:
 World Religion Day (2020-01-19) has been added to the holiday list.
Success:
 Robert E. Lee's Birthday (2020-01-19) has been adde

In [8]:
h = copy.deepcopy(hInit)

In [11]:
xmas = Holiday('birfday', date(2020,12,12))

In [12]:
h.addHoliday(xmas)

Success:
 birfday (2020-12-12) has been added to the holiday list.


In [14]:
xmas2 = Holiday('birfday', date(2020,12,11))

In [19]:
xmas2.date.day

11

In [15]:
hInit.addHoliday(xmas2)

Success:
 birfday (2020-12-11) has been added to the holiday list.


In [16]:
h == hInit

False

In [ ]:
h.read_json('example.json')

In [ ]:
h.innerHolidays

In [ ]:
h.numHolidays()

In [ ]:
h.scrapeHolidays()

In [ ]:
h.save_to_json('example.json')

In [ ]:
h.viewCurrentWeek()

In [ ]:
# if week_number == 1:
#     sorted(list(filter(lambda x: (x.date.isocalendar()[1] == week_number and x.date.year == year) , h.innerHolidays)), key = lambda x: x.date)
# elif week_number == 52:

myList = sorted(list(filter(lambda x: (x.date.isocalendar()[1] == week_number and x.date.year == year) , h.innerHolidays)), key = lambda x: x.date)
if week_number == 1:
    range = len(myList)
    i = 0
    while i < range:
        if myList[i].date.month == 12:
            removed = myList.pop(i)
            range -= 1
        i += 1
elif week_number == 52:
    range = len(myList)
    i = 0
    while i < range:
        if myList[i].date.month == 1:
            removed = myList.pop(i)
            range -= 1
        i += 1
            
print(myList)

In [ ]:
h.filter_holidays_by_week(2024,52)

In [ ]:
displaying = h.filter_holidays_by_week(2022,52)

In [ ]:
h.displayHolidaysInWeek(displaying)

In [ ]:
h1 = Holiday('myBirthday',date(2022,12,31))
h2 = Holiday('day after myBirthday',date(2023,1,1))

In [ ]:
print(sorted([h1,h2],key = lambda x: x.date))

In [ ]:
h.numHolidays()

In [ ]:
len([hol for hol in h.innerHolidays if hol.date.year == 2024])

In [ ]:
for hol in h.innerHolidays:
    if hol.date.year == 2024:
        print(hol,hol.date)

In [ ]:
def save_to_json(h, filelocation):

    with open(filelocation, 'w') as f:
        data = json.dump(self.innerHoliday, f)
    # Write out json file to selected file.

In [ ]:
def read_json(holidays, file):    
    with open(file, 'r') as f:
        data = json.load(f)
    for hol in data['holidays']:
        tmpDate = datetime.strptime(hol['date'],"%Y-%m-%d").date()
        tmpName = hol['name']
        tmpHoliday = Holiday(tmpName,tmpDate)
        # print((data['holidays'][0]))

In [ ]:
h = []
read_json(h, 'holidays.json')

In [ ]:
hInit = HolidayList()
h = d

In [ ]:
h.scrapeHolidays()

In [ ]:
h.innerHolidays

In [ ]:
h10 = Holiday('myBirthday',date(1999,12,1))

In [ ]:
h10.date.isocalendar()[1]

In [ ]:
h.addHoliday(h1)

In [ ]:
print(h.innerHolidays)

In [ ]:
fDate = datetime.strptime((hol['date'],"%Yb-%m-%d").date()

In [ ]:
data = [
    {'name':'Will','age':55},
    {'name':'Heather','age':54}
]

In [ ]:
data.sort(key=lambda item: item['name'])
data

In [4]:
def main():
    # 1. Initialize HolidayList Object
    h = HolidayList()

    # 2. Load JSON file via HolidayList read_json function
    h.read_json('holidays.json')

    # 3. Scrape additional holidays using your HolidayList scrapeHolidays function.
    h.scrapeHolidays()

    # 3. Create while loop for user to keep adding or working with the Calendar
    print('start_up') # turn into a variable for text pulled from config
    stillWorking = True
    while stillWorking:

    # 4. Display User Menu (Print the menu)
        print('main_menu') # turn into a variable for text pulled from config

    # 5. Take user input for their action based on Menu and check the user input for errors
        action = ''
        goodInput = False
        while not goodInput:
            action = input('Input a number[1-5]: ')
            action = action.strip()
            if action in ['1','2','3','4','5']:
                goodInput = True
            else:
                print(f'{action} is not a number 1-5')

    # 6. Run appropriate method from the HolidayList object depending on what the user input is
        if      action == '1':
            print('add_a_holiday')
            userHoliday = input('Holiday: ')
            userDate = ''
            goodInput = False
            while not goodInput:
                userDate = input(f'Date for {userHoliday}: ')
                userDate = userDate.strip()
                try:
                    userDate = datetime.strptime(userDate,"%Y-%m-%d").date()
                except ValueError:
                    print(f'Error: \n{userDate} is not a valid date. Please try again.')
                    continue
                else:
                    goodInput = True
            holToAdd = Holiday(userHoliday, userDate)
            h.addHoliday(holToAdd)

        elif    action == '2':
            print('remove_a_holiday')

        
        elif    action == '3':
            print('save_holiday_list')
            userSave = ''
            goodInput = False
            while not goodInput:
                userSave = input('Are you sure you want to save your changes? [y/n] ')
                userSave = userSave.strip().lower()
                if userSave in ['y','n']:
                    goodInput = True
                else:
                    print(f'{userSave} is not a valid input')
                if userSave == 'y':
                    h.save_to_json('holiday_list.json')
                    print('Success: \nYour changes have been saved.')
                else:
                    print('Canceled: \nHoliday list file save canceled')

        elif    action == '4':
            print('view_holidays')
        
        elif    action == '5':
            print("exit")
            if True: # if no changes have been made
                userExit = ''
                goodInput = False
                while not goodInput:
                    userExit = input('Are you sure you want to exit? [y/n] ')
                    userExit = userExit.strip().lower()
                    if userExit in ['y','n']:
                        goodInput = True
                    else:
                        print(f'{userExit} is not a valid input')
                if userExit == 'y':
                    print('Goodbye!')
                    stillWorking = False
            else: 
                userExit = ''
                goodInput = False
                while not goodInput:
                    userExit = input(
                    'Are you sure you want to exit? \nYour changes will be lost. \n[y/n] ')
                    userExit = userExit.strip().lower()
                    if userExit in ['y','n']:
                        goodInput = True
                    else:
                        print(f'{userExit} is not a valid input')
                if userExit == 'y':
                    print('Goodbye!')
                    stillWorking = False

    # 7. Ask the User if they would like to Continue, if not, end the while loop, ending the program.  If they do wish to continue, keep the program going. 


In [ ]:
userHoliday = 'xmas'
userDate = input(f'Date for {userHoliday}: ')
userDate = userDate.strip()


In [ ]:
userDate = '42-43-42'

In [ ]:
datetime.strptime(userDate,"%Y-%m-%d").date()

In [5]:
main()

Success:
 Margaret Thatcher Day (2021-01-10) has been added to the holiday list.
Success:
 World Sketchnote Day (2021-01-11) has been added to the holiday list.
Success:
 Zanzibar Revolution Day (2021-01-12) has been added to the holiday list.
Success:
 National Rubber Ducky Day (2021-01-13) has been added to the holiday list.
Success:
 Tamil Thai Pongdal Day (2021-01-14) has been added to the holiday list.
Success:
 National Bagel Day (2021-01-15) has been added to the holiday list.
Success:
 Signing of the Peace Accords (2021-01-16) has been added to the holiday list.
Success:
 New Year's Day (2020-01-01) has been added to the holiday list.
Success:
 World Braille Day (2020-01-04) has been added to the holiday list.
Success:
 Epiphany (2020-01-06) has been added to the holiday list.
Success:
 Asarah B'Tevet (2020-01-07) has been added to the holiday list.
Success:
 Orthodox Christmas Day (2020-01-07) has been added to the holiday list.
Success:
 International Programmers' Day (2020-0

UnboundLocalError: local variable 'userExit' referenced before assignment